In [135]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep # Esto hará que los request esperen dos segundos entre ellos dentro del 'While loop' para no llamar la atención o ser pillados.
import random                       #En esta página no hace falta porque esta hecha con fines educativos para 'Scraping'.

### Display the quote to the user and ask who said it. The player will have four guesses remaining.
### After each incorrect guess, the number of guesses remaining will decrement. If the player gets to zero guesses without identifying the author, the player loses and the game ends. If the player correctly identifies the author, the player wins!
### After every incorrect guess, the player receives a hint about the author:
##### For the first hint, make another request to the author's bio page (this is why we originally scrape this data), and tell the player the author's birth date and location.
##### The next two hints are up to you! Some ideas: the first letter of the author's first name, the first letter of the author's last name, the number of letters in one of the names, etc.
### When the game is over, ask the player if they want to play again. If yes, restart the game with a new quote. If no, the program is complete.


In [136]:
all_quotes = []
base_url = "https://quotes.toscrape.com/"  # Creamos esto para luego appendearle página2,3, etc..
url_paginas = "/page/1"    # Esta variable irá cambiando.

while url_paginas:    # Usamos este loop para obtener las "quotes" de todas las páginas.
    res_base = requests.get(f"{base_url}{url_paginas}")   # Hacemos que el request busque diferentes urls dependiendo de las páginas.
    print(f"Se está 'Scrapeando' {base_url}{url_paginas}")
    sopa = BeautifulSoup(res_base.text, "html.parser")
    quotes = sopa.find_all(class_= "quote") # La clase "quote" la encontré inspeccionando la página.
    
    for quote in quotes:    # Aqui damos la informacion a la variable all_quotes
        all_quotes.append({
        "frase" : quote.find(class_= "text").get_text(),  #get_text() limpia el texto eliminando los elemento
        "autor" : quote.find(class_="author").get_text(),
        "bio-enlace" : quote.find("a")["href"]              
        #Corchetes en "href"es syntax para elegir un solo atributo.
    })
        
    next_button = sopa.find(class_="next")
    url_paginas = next_button.find("a") ["href"] if next_button else None    # Este while hace que el url cambie hasta que no hayan más botones next en las páginas.
   # sleep(2)           # Esto hará que los request esperen dos segundos entre ellos para no llamar la atención o ser pillados.
   
   # En este punto (si fuera un juego real) podríamos guardar all_quotes en un csv, etc. e irlas actualizando cada semana (por ejemplo). Así todo irá más rápido.
   
   # Ahora empezamos a codificar el juego:
   
quote = random.choice(all_quotes)
intentos_maximos = 4
print("Quién es el autor de esta frase:")
print(quote["frase"])
print(quote["autor"])
adivinanza = ""
while adivinanza.lower() != quote["autor"].lower() and intentos_maximos > 0:
    adivinanza = input(f"Escribe tu respuesta. Te quedan {intentos_maximos} intentos").lower()
    if adivinanza == quote["autor"].lower():
        print("!Correcto!")
        break
    intentos_maximos -= 1
    if intentos_maximos == 3:   # Esto da una pista despues del primer intento, ya que el max de intento es 4.
        res = requests.get(f"{base_url}{quote['bio-enlace']}")
        sopa = BeautifulSoup(res.text,"html.parser")
        año_nacimiento = sopa.find(class_= "author-born-date").get_text()   # Esta clase la encontre inspeccionando la pagina de bio de  un autor.
        lugar_nacimiento = sopa.find(class_= "author-born-location").get_text()   # Esta clase la encontre inspeccionando la pagina de bio de  un autor.
        print(f"Te doy una pista: el autor de la frase nació {lugar_nacimiento} {año_nacimiento}")
    elif intentos_maximos == 2:
        print(f"Te doy otra pista: El nombre del autor de la frase empieza por la letra '{quote['autor'][0]}'")
    elif intentos_maximos == 1:
        inicial_apellido = quote['autor'].split(" ")[1][0] # Guardamos en variable porque tenemos que hacer un split() para dividir entre espacio en el string primero.
        print(f"Otra pista: El apellido del autor empieza por la letra {inicial_apellido}")
    else: print(f"Lo siento, no te quedan más intentos. La respuesta correcta es {quote['autor']}")
        
    
    
print("AFTER WHILE LOOP")
   

Se está 'Scrapeando' https://quotes.toscrape.com//page/1
Se está 'Scrapeando' https://quotes.toscrape.com//page/2/
Se está 'Scrapeando' https://quotes.toscrape.com//page/3/
Se está 'Scrapeando' https://quotes.toscrape.com//page/4/
Se está 'Scrapeando' https://quotes.toscrape.com//page/5/
Se está 'Scrapeando' https://quotes.toscrape.com//page/6/
Se está 'Scrapeando' https://quotes.toscrape.com//page/7/
Se está 'Scrapeando' https://quotes.toscrape.com//page/8/
Se está 'Scrapeando' https://quotes.toscrape.com//page/9/
Se está 'Scrapeando' https://quotes.toscrape.com//page/10/
Quién es el autor de esta frase:
“You can never get a cup of tea large enough or a book long enough to suit me.”
C.S. Lewis
!Correcto!
AFTER WHILE LOOP
